In [113]:
# suppress logging
import logging
logging.getLogger('jaclang').setLevel(logging.CRITICAL)

from jaclang.jac.workspace import Workspace
import os

jlws = Workspace(path='/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples')

In [114]:
modules = jlws.modules
for mod, mod_info in modules.items():
    print(mod, mod_info.ir.doc.value)

/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/circle_impl.jac """Enum for shape types"""
/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/guess_number.jac """A Number Guessing Game"""
/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/circle.jac """
This module demonstrates a simple circle class and a function to calculate
the area of a circle in all of Jac's glory.
"""


In [115]:
import jaclang.jac.absyntree as ast

file_url = list(modules.keys())[1]
print(file_url)
jlws.get_dependencies(file_url)

/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/guess_number.jac


In [116]:
file_url

'/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/guess_number.jac'

In [117]:
module = jlws.modules[file_url]

In [118]:
module.ir.sym_tab.kid

[GuessGame <jaclang.jac.symtable.SymbolTable object at 0x1102fce10>:
     start_game: Symbol(start_game, ability, public, None, [<jaclang.jac.absyntree.Ability object at 0x1102b5410>])
     process_guess: Symbol(process_guess, ability, public, None, [<jaclang.jac.absyntree.Ability object at 0x1102b7150>])
     correct_number: Symbol(correct_number, var, public, None, [<jaclang.jac.absyntree.HasVar object at 0x1102aa350>]),
 turn <jaclang.jac.symtable.SymbolTable object at 0x1102fec10>:
     check: Symbol(check, ability, public, None, [<jaclang.jac.absyntree.Ability object at 0x1102bc850>]),
 (n)turn.(a)check <jaclang.jac.symtable.SymbolTable object at 0x1102ffbd0>:
     guess: Symbol(guess, var, public, None, [<jaclang.jac.absyntree.Name object at 0x1102c1690>]),
 (w)GuessGame.(a)start_game <jaclang.jac.symtable.SymbolTable object at 0x110303c50>:
     end: Symbol(end, var, public, None, [<jaclang.jac.absyntree.Name object at 0x1102cc510>]),
 (w)GuessGame.(a)process_guess <jaclang.jac.

In [132]:
module.ir.sym_tab.tab['RAD'].sym_type == "var"

False

In [98]:
from jaclang.jac.symtable import SymbolTable, Symbol as JCSymbol
from jaclang.jac.absyntree import String, AstNode
from lsprotocol.types import Location, Range, Position, SymbolInformation, DocumentSymbol, SymbolKind
import os

class Symbol:
    def __init__(self, node: SymbolTable |AstNode , doc_uri: str, is_use: bool = False):
        if isinstance(node, SymbolTable):
            self.sym_tab = node
        self.is_use = is_use
        self.node = node.owner if isinstance(node, SymbolTable) else node
        self.doc_uri = doc_uri
        
    @property
    def sym_name(self):
        return self.node.sym_name

    @property 
    def ws_symbol(self):
        return self.node.sym_link

    @property
    def sym_type(self):
        return str(self.node.sym_type)

    @property
    def sym_doc(self):
        return self.ws_symbol.decl.doc.value[3:-3] if hasattr(self.ws_symbol.decl, 'doc') and isinstance(self.ws_symbol.decl.doc, String) else ""
    
    @property
    def defn_loc(self):
        return Location(
            uri=f"file://{os.path.join(os.getcwd(), self.ws_symbol.decl.loc.mod_path)}",
            range=Range(
                start= Position(
                    line=self.ws_symbol.decl.sym_name_node.loc.first_line - 1,
                    character=self.ws_symbol.decl.sym_name_node.loc.col_start - 1
                ),
                end= Position(
                    line=self.ws_symbol.decl.sym_name_node.loc.last_line - 1,
                    character=self.ws_symbol.decl.sym_name_node.loc.col_end - 1
                )
            )
        )

    @property
    def sym_info(self):
        return SymbolInformation(
            name=self.sym_name,
            kind=self._get_symbol_kind(self.sym_type),
            location=Location(
                uri=self.doc_uri,
                range=Range(
                    start = Position(
                        line=self.node.sym_name_node.loc.first_line - 1,
                        character=self.node.sym_name_node.loc.col_start - 1
                    ),
                    end = Position(
                        line=self.node.sym_name_node.loc.last_line - 1,
                        character=self.node.sym_name_node.loc.col_end - 1
                    )
                )
            )
        )

    @property
    def doc_sym(self):
        return DocumentSymbol(
            name=self.sym_name,
            kind=self.sym_info.kind,
            range=self.sym_info.location.range,
            selection_range=self.sym_info.location.range,
            detail=self.sym_doc,
            children=self._get_children_doc_sym()
        )

    @property
    def children(self):
        children = []
        if hasattr(self, "sym_tab"):
            for kid_sym_tab in self.sym_tab.kid:
                kid_symbol = Symbol(kid_sym_tab, self.doc_uri)
                children.append(kid_symbol)
        vars = (
            self.node.get_all_sub_nodes(HasVar)
            if isinstance(self.node, Architype)
            else self.node.get_all_sub_nodes(ParamVar)
            if isinstance(self.node, Ability)
            else []
        )
        for var in vars:
            var_symbol = Symbol(var, self.doc_uri)
            children.append(var_symbol)
        return children

    def _get_children_doc_sym(self):
        children = []
        for kid_symbol in self.children:
            try:
                children.append(kid_symbol.doc_sym)
            except Exception:
                pass
        return children

    @staticmethod
    def _get_symbol_kind(sym_type:str) -> SymbolKind:
        sym_type_map = {
            "mod": SymbolKind.Module,
            "mod_var": SymbolKind.Variable,
            "var": SymbolKind.Variable,
            "immutable": SymbolKind.Variable,
            "ability": SymbolKind.Function,
            "object": SymbolKind.Class,
            "node": SymbolKind.Class,
            "edge": SymbolKind.Class,
            "walker": SymbolKind.Class,
            "enum": SymbolKind.Enum,
            "test": SymbolKind.Function,
            "type": SymbolKind.TypeParameter,
            "impl": SymbolKind.Method,
            "field": SymbolKind.Field,
            "method": SymbolKind.Method,
            "constructor": SymbolKind.Constructor,
            "enum_member": SymbolKind.EnumMember,
        }
        return sym_type_map.get(sym_type, SymbolKind.Variable)

In [99]:
def get_doc_symbols(url) -> list[DocumentSymbol]:
    doc_symbols: list[DocumentSymbol] = []
    module = jlws.modules[url]
    
    for sym_tab in module.ir.sym_tab.kid:
        try:
            doc_symbols.append(Symbol(sym_tab, f"file://{url}").doc_sym)
        except:
            pass
    return doc_symbols

In [100]:
doc_syms = get_doc_symbols(file_url)

In [101]:
file_url

'/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/guess_number.jac'

In [102]:
module = jlws.modules[file_url]
module.ir.sym_tab.owner.kid[5]

In [103]:
doc_syms

[]

In [38]:
def pprint_doc_sym(doc_sym: DocumentSymbol, indent: int = 0):
    print(" " * indent + doc_sym.name)
    for child in doc_sym.children:
        pprint_doc_sym(child, indent + 2)

In [43]:
pprint_doc_sym(doc_syms[3])

(w)GuessGame.(a)start_game


In [15]:
jlws.get_definitions(file_url)

In [68]:
class Apple:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __init__(self, a):
        self.a = a
        self.b = 0

In [70]:
Apple(1).b

0

In [12]:
symbols = get_doc_symbols(file_url)

In [9]:
for s in symbols:
    print(s.node)

In [8]:
from common.symbols import Symbol

Symbol(f"file://{file_url}", symbols[0].children[2]).children

In [36]:
[x.value for x in jlws.get_definitions(file_url)[15].body.get_all_sub_nodes(ast.Name)].sym_link

AttributeError: 'list' object has no attribute 'sym_link'

In [8]:
for i in jlws.get_definitions("test.jac"):
    if hasattr(i, "doc") and isinstance(i.doc, ast.String):
        doc_str = i.doc.value
        doc_str = doc_str[3:-3]
        print(doc_str)

In [9]:
for mod_path, mod_info in jlws.modules.items():
    print(mod_path)

/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/circle_impl.jac
/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/guess_number.jac
/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/circle.jac


In [10]:
jlws.file_list()

['/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/circle_impl.jac',
 '/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/guess_number.jac',
 '/Users/chandralegend/Desktop/Jaseci/jaclang-vscode/examples/circle.jac']

In [11]:
jlws.get_symbols('test.jac')

[]

In [12]:
jlws.get_symbols('test.jac')[1]

IndexError: list index out of range

In [19]:
for symbol in jlws.get_symbols('test.jac'):
    print(symbol.sym_name)
    print(symbol.sym_type)
    print(symbol.decl.sym_name_node.loc)

random
module
3:11 - 3:17
circle_impl
module
4:13 - 4:24
GuessGame
type
6:8 - 6:17
turn
type
13:6 - 13:10
(n)turn.(a)check
impl
22:1 - 22:25
(w)GuessGame.(a)start_game
impl
34:1 - 34:37
(w)GuessGame.(a)process_guess
impl
44:1 - 44:40
start_game
ability
9:9 - 9:19
process_guess
ability
10:9 - 10:22
correct_number
var
7:9 - 7:23
guess
var
10:23 - 10:28
check
ability
14:9 - 14:14
guess
var
10:23 - 10:28


In [20]:
class Apple:
    name: str

x = Apple()

In [21]:
x.address = "123"

In [23]:
x.address

'123'

In [24]:
y = {}
y[x] = 1

In [26]:
y.keys()

dict_keys([<__main__.Apple object at 0x11627fe90>])

In [42]:
import time
async def wait_for():
    # wait 20 seconds and print "hello" without async io
    time.sleep(20)
    print("hello")

def main():
    wait_for()
    print("world")

In [43]:
await main()

world


/var/folders/b2/5v_f5db55cl2944tg8wh54w80000gn/T/ipykernel_54762/1230333363.py:8: RuntimeWarning: coroutine 'wait_for' was never awaited
  wait_for()


TypeError: object NoneType can't be used in 'await' expression

In [51]:
":walker:GuessGame:".replace(":", " ").split()[-2]

'walker'

In [63]:
import re

string = "Apple."
pattern = r"(\w+)."

match = re.match(pattern, string)
if match:
    architype = match.group(1)

    print(f"ARCHITYPE: {architype}")
else:
    print("Invalid string format")

ARCHITYPE: Apple
